In [1]:
%matplotlib notebook
%load_ext autoreload
%autoreload 1
%pwd

'/ocean/projects/asc170022p/mtragoza/lung-project'

In [2]:
import sys, os
import numpy as np
%aimport data

In [3]:
!tree --sort=mtime -L 1 data/Emory-4DCT

data/Emory-4DCT
├── Case1Pack
├── Case2Pack
├── Case3Pack
├── Case4Pack
├── Case5Pack
├── Case6Pack
├── Case7Pack
├── Case8Deploy
├── Case9Pack
├── Case10Pack
└── metadata.csv

10 directories, 1 file


In [4]:
!tree --sort=mtime data/Emory-4DCT/Case1Pack

data/Emory-4DCT/Case1Pack
├── Sampled4D
│   ├── case1_4D-75_T00.txt
│   ├── case1_4D-75_T50.txt
│   ├── case1_4D-75_T10.txt
│   ├── case1_4D-75_T20.txt
│   ├── case1_4D-75_T30.txt
│   └── case1_4D-75_T40.txt
├── ExtremePhases
│   ├── Case1_300_T00_xyz.txt
│   └── Case1_300_T50_xyz.txt
└── Images
    ├── case1_T00_s.img
    ├── case1_T10_s.img
    ├── case1_T20_s.img
    ├── case1_T30_s.img
    ├── case1_T40_s.img
    ├── case1_T50_s.img
    ├── case1_T60_s.img
    ├── case1_T70_s.img
    ├── case1_T80_s.img
    └── case1_T90_s.img

3 directories, 18 files


In [141]:
%autoreload
ds = data.Emory4DCTDataset('data/Emory-4DCT')
ds

Emory4DCTDataset([
  Emory4DCTCase('data/Emory-4DCT', 'Case1Pack', 10 phases),
  Emory4DCTCase('data/Emory-4DCT', 'Case2Pack', 10 phases),
  Emory4DCTCase('data/Emory-4DCT', 'Case3Pack', 10 phases),
  Emory4DCTCase('data/Emory-4DCT', 'Case4Pack', 10 phases),
  Emory4DCTCase('data/Emory-4DCT', 'Case5Pack', 10 phases),
  Emory4DCTCase('data/Emory-4DCT', 'Case6Pack', 10 phases),
  Emory4DCTCase('data/Emory-4DCT', 'Case7Pack', 10 phases),
  Emory4DCTCase('data/Emory-4DCT', 'Case8Deploy', 10 phases),
  Emory4DCTCase('data/Emory-4DCT', 'Case9Pack', 10 phases),
  Emory4DCTCase('data/Emory-4DCT', 'Case10Pack', 10 phases)
])

In [142]:
ds.load_images()

Loading data/Emory-4DCT/Case1Pack/Images/case1_T00_s.img
Loading data/Emory-4DCT/Case1Pack/Images/case1_T10_s.img
Loading data/Emory-4DCT/Case1Pack/Images/case1_T20_s.img
Loading data/Emory-4DCT/Case1Pack/Images/case1_T30_s.img
Loading data/Emory-4DCT/Case1Pack/Images/case1_T40_s.img
Loading data/Emory-4DCT/Case1Pack/Images/case1_T50_s.img
Loading data/Emory-4DCT/Case1Pack/Images/case1_T60_s.img
Loading data/Emory-4DCT/Case1Pack/Images/case1_T70_s.img
Loading data/Emory-4DCT/Case1Pack/Images/case1_T80_s.img
Loading data/Emory-4DCT/Case1Pack/Images/case1_T90_s.img
Loading data/Emory-4DCT/Case2Pack/Images/case2_T00-ssm.img
Loading data/Emory-4DCT/Case2Pack/Images/case2_T10-ssm.img
Loading data/Emory-4DCT/Case2Pack/Images/case2_T20-ssm.img
Loading data/Emory-4DCT/Case2Pack/Images/case2_T30-ssm.img
Loading data/Emory-4DCT/Case2Pack/Images/case2_T40-ssm.img
Loading data/Emory-4DCT/Case2Pack/Images/case2_T50-ssm.img
Loading data/Emory-4DCT/Case2Pack/Images/case2_T60-ssm.img
Loading data/Emor

In [143]:
ds.save_niftis()

Saving data/Emory-4DCT/Case1Pack/NIFTI/case1_T00.nii.gz
Saving data/Emory-4DCT/Case1Pack/NIFTI/case1_T10.nii.gz
Saving data/Emory-4DCT/Case1Pack/NIFTI/case1_T20.nii.gz
Saving data/Emory-4DCT/Case1Pack/NIFTI/case1_T30.nii.gz
Saving data/Emory-4DCT/Case1Pack/NIFTI/case1_T40.nii.gz
Saving data/Emory-4DCT/Case1Pack/NIFTI/case1_T50.nii.gz
Saving data/Emory-4DCT/Case1Pack/NIFTI/case1_T60.nii.gz
Saving data/Emory-4DCT/Case1Pack/NIFTI/case1_T70.nii.gz
Saving data/Emory-4DCT/Case1Pack/NIFTI/case1_T80.nii.gz
Saving data/Emory-4DCT/Case1Pack/NIFTI/case1_T90.nii.gz
Saving data/Emory-4DCT/Case2Pack/NIFTI/case2_T00.nii.gz
Saving data/Emory-4DCT/Case2Pack/NIFTI/case2_T10.nii.gz
Saving data/Emory-4DCT/Case2Pack/NIFTI/case2_T20.nii.gz
Saving data/Emory-4DCT/Case2Pack/NIFTI/case2_T30.nii.gz
Saving data/Emory-4DCT/Case2Pack/NIFTI/case2_T40.nii.gz
Saving data/Emory-4DCT/Case2Pack/NIFTI/case2_T50.nii.gz
Saving data/Emory-4DCT/Case2Pack/NIFTI/case2_T60.nii.gz
Saving data/Emory-4DCT/Case2Pack/NIFTI/case2_T70

In [145]:
%autoreload
ds.load_niftis()

Loading data/Emory-4DCT/Case1Pack/NIFTI/case1_T00.nii.gz
Loading data/Emory-4DCT/Case1Pack/NIFTI/case1_T10.nii.gz
Loading data/Emory-4DCT/Case1Pack/NIFTI/case1_T20.nii.gz
Loading data/Emory-4DCT/Case1Pack/NIFTI/case1_T30.nii.gz
Loading data/Emory-4DCT/Case1Pack/NIFTI/case1_T40.nii.gz
Loading data/Emory-4DCT/Case1Pack/NIFTI/case1_T50.nii.gz
Loading data/Emory-4DCT/Case1Pack/NIFTI/case1_T60.nii.gz
Loading data/Emory-4DCT/Case1Pack/NIFTI/case1_T70.nii.gz
Loading data/Emory-4DCT/Case1Pack/NIFTI/case1_T80.nii.gz
Loading data/Emory-4DCT/Case1Pack/NIFTI/case1_T90.nii.gz
Loading data/Emory-4DCT/Case2Pack/NIFTI/case2_T00.nii.gz
Loading data/Emory-4DCT/Case2Pack/NIFTI/case2_T10.nii.gz
Loading data/Emory-4DCT/Case2Pack/NIFTI/case2_T20.nii.gz
Loading data/Emory-4DCT/Case2Pack/NIFTI/case2_T30.nii.gz
Loading data/Emory-4DCT/Case2Pack/NIFTI/case2_T40.nii.gz
Loading data/Emory-4DCT/Case2Pack/NIFTI/case2_T50.nii.gz
Loading data/Emory-4DCT/Case2Pack/NIFTI/case2_T60.nii.gz
Loading data/Emory-4DCT/Case2Pa

In [146]:
case = ds.cases[0]
case.view(groupby=['phase'], xlim=(-1000, 15000), ylim=(0, 4e6), bins=np.linspace(0, 15000, 20))

:DynamicMap   [phase]
   :NdOverlay   [Element]
      :Histogram   [case1]   (case1_count)

In [147]:
case.view(groupby=['phase', 'y'], x='x', y='z')

:DynamicMap   [phase,y]
   :Image   [x,z]   (case1)

In [149]:
%autoreload
case.select(y=120).view(groupby='phase', x='x', y='z', widget_type='scrubber', widget_location='bottom')

Column
    [0] HoloViews(DynamicMap, sizing_mode='fixed', widget_location='bottom', widget_type='scrubber')
    [1] WidgetBox(align=('center', 'end'))
        [0] Player(end=9, width=550)